## Test Neuroevolución

In [2]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler  

In [3]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import pickle
from copy import deepcopy

Importación del scaler
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

### Cargo los datos y retiro el Timestamp y el score

In [4]:
df = pd.read_csv("gameStates.csv", sep=',', header=None)
df.columns = ['timeStamp','Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6','ray1','ray2','ray3','ray4','ray5','ray6','ray7','ray8','ray9','ray10','ray11','ray12','ray13','ray14','ray15','ray16','ray17','ray18','ray19','ray20','ray21','ray22','ray23','ray24','ray25','ray26','ray27','score',"VKey","HKey","Shooting"]
df.drop(['timeStamp','score', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6'],axis=1,inplace=True)
df = df[1:6001]


#### Transforma el porblema multioputput en multilabel

In [5]:
def multioutput2multilabel(row):
    values = pd.Series([(row["VKey"] == "UpArrow"),(row["VKey"] == "DownArrow"),(row["HKey"] == "LeftArrow"),(row["HKey"] == "RightArrow"),row["Shooting"]])
    return pd.concat([row[:len(row)-3], values.astype(float)])


In [6]:
def concatenar_estados(dataframe):
        dfAnterior = dataframe.shift(1)
        dfAnterior.rename(columns=lambda x: str(x)+"-1", inplace=True)
        dfAnterior2 = dataframe.shift(2)
        dfAnterior2.rename(columns=lambda x: str(x)+"-2", inplace=True)
        dfAnterior3 = dataframe.shift(3)
        dfAnterior3.rename(columns=lambda x: str(x)+"-3", inplace=True)
        df = pd.concat([dfAnterior3,dfAnterior2,dfAnterior,dataframe],axis=1)
        df = df.fillna(999)
        return df

df = df.apply(multioutput2multilabel,axis=1)
len(df.ix[1])

39

In [7]:
df = concatenar_estados(df)
len(df.ix[1])

156

In [8]:
#Defino los elementos train_data y target_data
train = df.drop([0, 1, 2,3,4],axis=1).values
labels = df[[0,1,2,3,4]].values

In [9]:
#define el perceptrón y lo entrena
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(15,), random_state=1)
clf.fit(train, labels)
model = deepcopy(clf)
pickle.dump(clf, open(".\\AI_Data\\clf.sav", 'wb'))
pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]

In [10]:
#Neuroevolución

def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))


print(sizes)

[2265, 75]


### Pipeline

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


estimators = [('stantandarize', StandardScaler()), ('clf', MLPClassifier())]

pipe = Pipeline(estimators)

# entreno el pipe como si fuera un clasificador, 
# también podría hacer validación cruzada y todo como si fuese un clasificador normal
pipe.fit(train, labels)
pickle.dump(model, open(".\\AI_Data\\pipe.sav", 'wb'))


#from sklearn.model_selection import cross_val_score
#print(cross_val_score(pipe, train, labels, cv=5).mean())

C:\Users\Pablo Alejos\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [12]:


def MLPFitness(individual):
    coefs = gen2Coefs(np.array(individual),sizes,shapes)
    pipe.named_steps['clf'].coefs_ = coefs 
    pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
    os.system("AI.exe")
    scores = pd.read_csv("myData.csv", sep=',', header=None)
    os.remove("myData.csv")
    f = np.mean(scores.values)
    return (f,)


MLPFitness(coefs2gen(clf.coefs_,sizes,shapes))

(10.800000000000001,)

In [14]:
def initPopulation(pop, ind_random, ind_guess,n_guess=1,n=10):
    
    pop = []
    n_random = n - n_guess
    for i in range(n_random):
        pop.append(ind_random())
        
    for i in range(n_guess):
        pop.append(ind_guess())
    
    return pop

In [16]:
N = 4

pesosIniciales = coefs2gen(pipe.named_steps['clf'].coefs_,sizes,shapes).copy()
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]


creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximiza
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=sum(sizes))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("individual_guess", lambda :creator.Individual(pesosIniciales))
toolbox.register("population_mix",initPopulation,list,toolbox.individual, toolbox.individual_guess)       


toolbox.register("evaluate", MLPFitness)
toolbox.register("mate", tools.cxOnePoint)

toolbox.register("mutate", tools.mutGaussian, mu=0.5, sigma=0.5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=10)

random.seed(64)
  
population = toolbox.population_mix(n=10, n_guess=N)  
    
hof = tools.HallOfFame(1, similar=np.array_equal)
    
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
    
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, stats=stats, halloffame=hof)
print(hof) # hall of the fame contiene el mejor individuo vivo en cada generacion
mejorInd = hof[0] # esta ordenado de manera que el primer elemento es el mejor de siempre

MLPFitness(mejorInd)

gen	nevals	avg    	std    	min   	max 
0  	10    	-100.72	77.5475	-235.4	-8.2
1  	10    	-127.075	120.905	-338  	49.25
2  	3     	-3.93   	78.2636	-159.8	49.25
3  	8     	-100.465	100.322	-254.8	49.25
4  	4     	-28.24  	95.964 	-194.25	49.25
5  	5     	-25.49  	78.768 	-167.2 	49.25
[Individual([ 0.60502289,  0.50982951,  0.52769339, ...,  0.47431357,
        0.92370547,  0.92157749])]


(-48.0,)

In [17]:
MLPFitness(mejorInd)

(24.600000000000001,)

In [18]:
MLPFitness(mejorInd)

(-87.0,)

In [ ]:
### Calibrado de parámetros